# Spectral Timing In Stingray

This tutorial shows how to make and manipulate a cross spectrum of two light curves using Stingray.

In [1]:
import numpy as np
from stingray import Lightcurve, Crossspectrum, AveragedCrossspectrum

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
%matplotlib inline
font_prop = font_manager.FontProperties(size=16)

/Users/danielahuppenkothen/work/repositories/stingraysoftware/stingray/stingray/crossspectrum.py:21: UserWarning: Using standard scipy fft
  warnings.warn("Using standard scipy fft")


## Loading the Data

Let's first load the data. There's a text file in this repo with a binned X-ray light curve with three columns: the mid-point of each time bin, the associated photon counts per bin in energy range 1, and the associated photon counts per bin in energy range 2. Since it's a text file, and well-behaved, we can use `np.loadtxt` to load the data:

Let's now make light curves and plot the result:

## 2. Pass both of the light curves to the `Crossspectrum` class to create a `Crossspectrum` object.
The first `Lightcurve` passed is the channel of interest or interest band, and the second `Lightcurve` passed is the reference band.
You can also specify the optional attribute `norm` if you wish to normalize the real part of the cross spectrum to squared fractional rms, Leahy, or squared absolute normalization. The default normalization is 'none'.

We can print the first five values in the arrays of the positive Fourier frequencies and the cross power. The cross power has a real and an imaginary component.

Since the negative Fourier frequencies (and their associated cross powers) are discarded, the number of time bins per segment `n` is twice the length of `freq` and `power`.

# Properties
A `Crossspectrum` object has the following properties :

1. `freq` : Numpy array of mid-bin frequencies that the Fourier transform samples.
2. `power` : Numpy array of the cross spectrum (complex numbers).
3. `df` : The frequency resolution.
4. `m` : The number of cross spectra averaged together. For a `Crossspectrum` of a single segment, `m=1`.
5. `n` : The number of data points (time bins) in one segment of the light curves.
6. `nphots1` : The total number of photons in the first (interest) light curve.
7. `nphots2` : The total number of photons in the second (reference) light curve.

We can compute the amplitude of the cross spectrum, and plot it as a function of Fourier frequency. Notice how there's a spike at our signal frequency of 2 Hz!

You'll notice that the cross spectrum is a bit noisy. This is because we're only using one segment of data. Let's try averaging together multiple segments of data.
# Averaged cross spectrum example
You could use two long `Lightcurve`s and have `AveragedCrossspectrum` chop them into specified segments, or give two lists of `Lightcurve`s where each segment of `Lightcurve` is the same length. We'll show the first way here. Remember to check the Lightcurve tutorial notebook for fancier ways of making light curves.

## Pass both light curves to the `AveragedCrossspectrum` class with a specified `segment_size`.
If the exposure (length) of the light curve cannot be divided by `segment_size` with a remainder of zero, the last incomplete segment is thrown out, to avoid signal artefacts. Here we're using 8 second segments.

Again we can print the first five Fourier frequencies and first five cross spectral values, as well as the number of segments.

If `m` is less than 50 and you try to compute the coherence, a warning will pop up letting you know that your number of segments is significantly low, so the error on `coherence` might not follow the expected (Gaussian) statistical distributions.

# Properties
An `AveragedCrossspectrum` object has the following properties, same as `Crossspectrum` :

1. `freq` : Numpy array of mid-bin frequencies that the Fourier transform samples.
2. `power` : Numpy array of the averaged cross spectrum (complex numbers).
3. `df` : The frequency resolution (in Hz).
4. `m` : The number of cross spectra averaged together, equal to the number of whole segments in a light curve.
5. `n` : The number of data points (time bins) in one segment of the light curves.
6. `nphots1` : The total number of photons in the first (interest) light curve.
7. `nphots2` : The total number of photons in the second (reference) light curve.

Let's plot the amplitude of the averaged cross spectrum!

Now we'll show examples of all the things you can do with a `Crossspectrum` or `AveragedCrossspectrum` object using built-in stingray methods.

# Normalizating the cross spectrum
The three kinds of normalization are:
* `leahy`: Leahy normalization. Makes the Poisson noise level $= 2$. See *Leahy et al. 1983, ApJ, 266, 160L*. 
* `frac`: Fractional rms-squared normalization, also known as rms normalization. Makes the Poisson noise level $= 2 / \sqrt(meanrate_1\times meanrate_2)$. See *Belloni & Hasinger 1990, A&A, 227, L33*, and *Miyamoto et al. 1992, ApJ, 391, L21.*
* `abs`: Absolute rms-squared normalization, also known as absolute normalization. Makes the Poisson noise level $= 2 \times \sqrt(meanrate_1\times meanrate_2)$. See *insert citation*.
* `none`: No normalization applied. This is the default.

Note that these normalizations and the Poisson noise levels apply to the "cross power", not the cross-spectral amplitude.

Here we plot the three normalized averaged cross spectra.

# Re-binning a cross spectrum in frequency
Typically, rebinning is done on an averaged, normalized cross spectrum.
## 1. We can linearly re-bin a cross spectrum
(although this is not done much in practice)

## 2. And we can logarithmically/geometrically re-bin a cross spectrum
In this re-binning, each bin size is 1+f times larger than the previous bin size, where `f` is user-specified and normally in the range 0.01-0.1. The default value is `f=0.01`.

Logarithmic rebinning only keeps the real part of the cross spectum.

Note that like `rebin`, `rebin_log` returns a `Crossspectrum` or `AveragedCrossspectrum` object (depending on the input object):

# Time lags / phase lags
## 1. Frequency-dependent lags
The lag-frequency spectrum shows the time lag between two light curves (usually non-overlapping broad energy bands) as a function of Fourier frequency.
See  *Uttley et al. 2014, A&ARev, 22, 72* section 2.2.1.

The `time_lag` method returns an `np.ndarray` with the time lag in seconds per positive Fourier frequency.

And this is a plot of the lag-frequency spectrum.

# Coherence
Coherence is a Fourier-frequency-dependent measure of the linear correlation between time series measured simultaneously in two energy channels. 
See *Vaughan and Nowak 1997, ApJ, 474, L43* and *Uttley et al. 2014, A&ARev, 22, 72* section 2.1.3. 

The `coherence` method returns two `np.ndarray`s, of the coherence and uncertainty.

And we can plot the coherence vs the frequency.

## Lag-Energy Spectra and Other Products

Lag-energy spectra and other, similar products (e.g. rms-energy spectra and covariance-energy spectra) are under construction. They are essentially implemented, but not well-documented. If you're interested in using them, please reach out to us! If you want to help us improve and document them while you're at it, you'd be our new best friend. :)

## Exercise: X-ray Binary or AGN?

I've simulated a light curve for you! Your task is to find out, from its spectral timing properties, whether the source is likely an X-ray binary or an AGN. 

Note: One (easy) way to distinguish AGN from XRBs are the timescales: because of the smaller mass and size, XRBs tend to vary on much shorter timescales. However, for this exercise, assume that the absolute time units don't matter; they are simulated to be arbitrary. You'll have to investigate the spectral timing properties to find out the truth. :) 